In [1]:
import pandas as pd
import numpy as np
from sympy import symbols, solve
from queue import Queue
from math import e
from scipy.optimize import fsolve

In [2]:
class Node:
    
    def __init__(self, data):
        self.data = data
        self.up = None
        self.down = None

In [3]:
df_vol = pd.read_excel("Homework_6_Data.xlsx", sheet_name=0)
df_d = pd.read_excel("Homework_6_Data.xlsx", sheet_name=1)
df = pd.merge(df_vol, df_d, on="T", how="inner")
df["Yield"] = (1/df["D(T)"]) ** (1/(df["T"]*2)) - 1
for i in range(1,len(df)):
    df.at[i,"Forward"] = ((1+df.at[i,"Yield"]) ** (i+1))/((1+df.at[i-1,"Yield"]) ** i)
    df.at[i,"Forward"] -= 1
df.at[0,"Forward"] = df.at[0,"Yield"]
df

,T,Sigma,D(T),Yield,Forward
0,0.5,0.100,0.972476,0.028302,0.028302
1,1.0,0.120,0.944569,0.028924,0.029545
2,1.5,0.135,0.916324,0.029557,0.030825
3,2.0,0.150,0.887934,0.030160,0.031973
4,2.5,0.160,0.859741,0.030686,0.032793
5,3.0,0.162,0.832044,0.031119,0.033288
6,3.5,0.164,0.805077,0.031459,0.033496
7,4.0,0.162,0.778904,0.031726,0.033602
8,4.5,0.160,0.753480,0.031950,0.033742
9,5.0,0.157,0.728734,0.032151,0.033958


In [24]:
def bootstrap(df):
    
    # Assume mean reversion as variable, yield is our best guess
    m_est = [df.at[i, "Yield"] for i in range(len(df))]   
        
    #Calculate m
    m = fsolve(eval_m, x0=m_est, args=(df))
        
    # Fill value of m in dataframe and repeat
    df["m"] = m
        
def eval_cf(start, end, dfact, r_tree, sigma, sqrt_dt):
    if start == end:
        return dfact
    
    ru = r_tree[start]
    rd = ru / (e ** (2 * sigma[i] * sqrt_dt))
    
    cfu = 0.5 * (1 + ru) * eval_cf(start+1, end, dfact, r_tree, sigma, sqrt_dt)
    cfd = 0.5 * (1 + rd) * eval_cf(start+1, end, dfact, r_tree, sigma, sqrt_dt)
    
    return cfu + cfd

def eval_m(m, df):
    
    dt = 0.5
    sqrt_dt = 0.5 ** 0.5
    r_tree = [df.at[0, "Yield"]] # will store only the r* the top element of the tree
    m_calc = []
    cf = 0
    
    # Calculate ith branch of interest rate tree using assumed m 
    for i in range(len(df)):
        sigma = df.at[i, "Sigma"]
        dfact = df.at[i, "D(T)"]
        r_tree.append(r_tree[i-1] * (e ** (m[i] * dt + sigma * sqrt_dt)))
        
        # Calculate ith branch of cash flow tree and evaluate expression to 1 to get value of m    
        cf = eval_cf(0, i, dfact, r_tree, df["Sigma"], sqrt_dt)
        m_calc.append(cf - 1)
        #print("Done")
    
    return m_calc
        
bootstrap(df)
df

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


KeyboardInterrupt: 

In [ ]:
df["BDT E(r)"] = 0
def level_order(root, i, df):
    if root == None:
        return
    df.at[i, "BDT E(r)"] = df.at[i, "BDT E(r)"] + (root.data)/(2 ** i)
    level_order(root.up, i+1, df)
    level_order(root.down, i+1, df)
level_order(root, 0, df)
df

In [ ]:
def height(root):
    if root == None:
        return 0
    else:
        return 1 + max(height(root.up), height(root.down))
height(root)

In [ ]:
df